In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Read in each of the data frames with the data for the plot
comb_dat = pd.read_csv('samples.csv')
comb_dat.loc[pd.isnull(comb_dat["name"]), "name"] = None
comb_dat.loc[pd.isnull(comb_dat["species"]), "species"] = 'Metagenomic'
comb_dat.loc[comb_dat['species'] == 'Metagenomic', "color"] = "#7C1B96"
comb_dat.loc[pd.notnull(comb_dat["name"]), "color"] = "#EE9F22"  # pathomap
comb_dat.loc[comb_dat['species'] == 'Bacillus thuringiensis', "color"] = "#330ECD"
comb_dat.loc[comb_dat['species'] == 'Bacillus anthracis', "color"] = "#9A1922"
comb_dat.loc[comb_dat['species'] == 'Bacillus cereus', "color"] = "#4C8ECD"
comb_dat.loc[comb_dat['species'] == 'Bacillus mycoides', "color"] = "#40C5CD"
comb_dat.loc[comb_dat['species'] == 'Bacillus subtilis', "color"] = "#00B1CD"

In [ ]:
import mpld3
from mpld3 import plugins
mpld3.enable_notebook()

In [ ]:
import matplotlib
from mpld3.utils import get_id

class PointHTMLExtendedTooltip(plugins.PluginBase):
    """Extension of PointHTMLTooltip from mpld3
    """

    JAVASCRIPT = """
    mpld3.register_plugin("htmltooltip", HtmlTooltipPlugin);
    HtmlTooltipPlugin.prototype = Object.create(mpld3.Plugin.prototype);
    HtmlTooltipPlugin.prototype.constructor = HtmlTooltipPlugin;
    HtmlTooltipPlugin.prototype.requiredProps = ["id"];
    HtmlTooltipPlugin.prototype.defaultProps = {labels:null,
                                                hoffset:0,
                                                voffset:10,
                                                ext_links:null};
    function HtmlTooltipPlugin(fig, props){
        mpld3.Plugin.call(this, fig, props);
    };
    HtmlTooltipPlugin.prototype.draw = function(){
       var obj = mpld3.get_element(this.props.id);
       var labels = this.props.labels;
       var ext_links = this.props.ext_links;
       var tooltip = d3.select("body").append("div")
                    .attr("class", "mpld3-tooltip")
                    .style("position", "absolute")
                    .style("z-index", "10")
                    .style("visibility", "hidden");
       obj.elements()
           .on("mousedown", function(d, i){
                               window.open(ext_links[i], "_blank");})
           .on("mouseover", function(d, i){
                              tooltip.html(labels[i])
                                     .style("visibility", "visible");})
           .on("mousemove", function(d, i){
                  tooltip
                    .style("top", d3.event.pageY + this.props.voffset + "px")
                    .style("left",d3.event.pageX + this.props.hoffset + "px");
                 }.bind(this))
           .on("mouseout",  function(d, i){
                           tooltip.style("visibility", "hidden");});
    };
    """

    def __init__(self, points, labels=None, ext_links=None,
                 hoffset=0, voffset=10, css=None):
        self.points = points
        self.labels = labels
        self.ext_links = ext_links
        self.voffset = voffset
        self.hoffset = hoffset
        self.css_ = css or ""
        if isinstance(points, matplotlib.lines.Line2D):
            suffix = "pts"
        else:
            suffix = None
        self.dict_ = {"type": "htmltooltip",
                      "id": get_id(points, suffix),
                      "labels": labels,
                      "hoffset": hoffset,
                      "voffset": voffset,
                      "ext_links": ext_links}

In [ ]:
# Add a nice custom label to display on rollover
comb_dat["label"] = None
for ix, vals in comb_dat.iterrows(): 
    comb_dat.loc[ix, "label"] = ("%s" % (vals["name"] or vals["species"]))

In [ ]:
comb_dat["xvals"] = comb_dat["bacillus_depth"].apply(np.log10)
comb_dat["yvals"] = comb_dat["mean_depth"].apply(np.log10)

In [ ]:
comb_dat.loc[comb_dat["xvals"] == -np.inf, "xvals"] = -4.0  # We need to do this bc matplotlib d/n handle -inf properly
comb_dat.loc[comb_dat["yvals"] == -np.inf, "yvals"] = -4.0

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(axisbg='#EEEEEE'))
ax.grid(color='white', linestyle='solid')

slope = 0.001
ax.plot(np.log10([0.00001, 1000000]), np.log10([slope * 0.00001, slope * 1000000]), linewidth=2.0,
        c="gray", zorder=1)

scatter = ax.scatter(x=comb_dat["xvals"],
                     y=comb_dat["yvals"],
                     c=comb_dat["color"],
                     s=100,
                     alpha=0.7,
                     zorder=2)

ax.set_title("Detection of B. anthracis in mixed samples", size=18);
ax.set_xlabel('Bacillus depth (log10)')
ax.set_ylabel('B. anthracis depth (log10)')
ax.set_xlim(0.48, 3.1)
ax.set_ylim(-4.1, 2.8)

ax.text(2.5, -0.15, '1:1000', fontsize=16, color="gray")

labels_html = ['<div class="nbtooltip">%s<br><em>B. anthracis</em> depth: %.2f<br><em>Bacillus</em> depth: %.2f</div>' % 
               (vals["label"], vals["mean_depth"], vals["bacillus_depth"]) for ix, vals in comb_dat.iterrows()]
ext_links = ["https://app.onecodex.com/markerpanel/public/%s" % (x if pd.notnull(x) else '') for x in comb_dat["uuid"]]
plugins.connect(fig, PointHTMLExtendedTooltip(scatter, labels=labels_html,
                                              ext_links=ext_links,
                                              hoffset=5, voffset=15,
                                              css=".mpld3-figure path {cursor: pointer;}"))
 

In [ ]:
fig.set_size_inches(8, 8)

In [ ]:
mpld3.display()

In [ ]:
with open('interactive_plot.html', 'w') as fo:
    fo.write(mpld3.fig_to_html(fig, template_type="simple"))